# HW 2: Deep Bag-of-Words

![words.jpeg](https://cdn-images-1.medium.com/max/1600/0*JpqZhCNsQ_OGaRkB.jpg)

<br>

In this homework, you will be implementing a deep averaging network, detailed in [Deep Unordered Composition  Rivals Syntactic Methods for Text Classification by Iyyer et al. (2015)](https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf) and training it to do sentiment analysis on the Stanford Sentiment Treebank.

<br>

**Please use all of the starter code that is provided, do not make any changes to the data processing, evaluation, and training functions. Only add code were you're asked to.**

<br>

# Part 0: Read Paper!

Read [Deep Unordered Composition  Rivals Syntactic Methods for Text Classification by Iyyer et al. (2015)](https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf).

## Data Processing

Make sure you've downloaded the Stanford Sentiment Treebank that was used in lab. You can find it [here](http://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip).

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import re
import random

random.seed(1)
sst_home = ''

# Let's do 2-way positive/negative classification instead of 5-way
easy_label_map = {0:0, 1:0, 2:None, 3:1, 4:1}
    # so labels of 0 and 1 in te 5-wayclassificaiton are 0 in the 2-way. 3 and 4 are 1, and 2 is none
    # because we don't have a neautral class. 

PADDING = "<PAD>"
UNKNOWN = "<UNK>"
max_seq_length = 20

def load_sst_data(path):
    data = []
    with open(path) as f:
        for i, line in enumerate(f): 
            example = {}
            example['label'] = easy_label_map[int(line[1])]
            if example['label'] is None:
                continue
            
            # Strip out the parse information and the phrase labels---we don't need those here
            text = re.sub(r'\s*(\(\d)|(\))\s*', '', line)
            example['text'] = text[1:]
            data.append(example)

    random.seed(1)
    random.shuffle(data)
    return data
     
training_set = load_sst_data('train.txt')
dev_set = load_sst_data('dev.txt')
test_set = load_sst_data('test.txt')

<br>

Next, we'll extract the vocabulary from the data, index each token, and finally convert the sentences into lists of indexed tokens. We are also padding and truncating all sentences to be of length=20. (Why? Think about how to handle batching. This is certainly not the only way! This is just simple.)

In [0]:
import collections
import numpy as np

def tokenize(string):
    return string.split()

def build_dictionary(training_datasets):
    """
    Extract vocabulary and build dictionary.
    """  
    word_counter = collections.Counter()
    for i, dataset in enumerate(training_datasets):
        for example in dataset:
            word_counter.update(tokenize(example['text']))
        
    vocabulary = set([word for word in word_counter])
    vocabulary = list(vocabulary)
    vocabulary = [PADDING, UNKNOWN] + vocabulary
        
    word_indices = dict(zip(vocabulary, range(len(vocabulary))))

    return word_indices, len(vocabulary)

def sentences_to_padded_index_sequences(word_indices, datasets):
    """
    Annotate datasets with feature vectors. Adding right-sided padding. 
    """
    for i, dataset in enumerate(datasets):
        for example in dataset:
            example['text_index_sequence'] = torch.zeros(max_seq_length)

            token_sequence = tokenize(example['text'])
            padding = max_seq_length - len(token_sequence)

            for i in range(max_seq_length):
                if i >= len(token_sequence):
                    index = word_indices[PADDING]
                    pass
                else:
                    if token_sequence[i] in word_indices:
                        index = word_indices[token_sequence[i]]
                    else:
                        index = word_indices[UNKNOWN]
                example['text_index_sequence'][i] = index

            example['text_index_sequence'] = example['text_index_sequence'].long().view(1,-1)
            example['label'] = torch.LongTensor([example['label']])


word_to_ix, vocab_size = build_dictionary([training_set])
sentences_to_padded_index_sequences(word_to_ix, [training_set, dev_set, test_set])

In [0]:
print("Size of training dataset:", len(training_set))
print("\nFirst padded and indexified example in training data:\n", training_set[0])

('Size of training dataset:', 6920)
('\nFirst padded and indexified example in training data:\n', {'text': "Resident Evil is n't a product of its cinematic predecessors so much as an MTV , sugar hysteria , and PlayStation cocktail .", 'text_index_sequence': tensor([[14711,  1967,  8638,  9667,  6138,  5124, 13993,  2189,   431, 10746,
          1200,   609, 11730,  1890, 15979,  8186,  8453, 16253,  8186, 14190]]), 'label': tensor([0])})


<br>
### Batichify data
We're going to be doign mini-batch training. The following code makes data iterators and a batchifying function.

In [0]:
# This is the iterator we'll use during training. 
# It's a generator that gives you one batch at a time.
def data_iter(source, batch_size):
    dataset_size = len(source)
    start = -1 * batch_size
    order = list(range(dataset_size))
    random.shuffle(order)

    while True:
        start += batch_size
        if start > dataset_size - batch_size:
            # Start another epoch.
            start = 0
            random.shuffle(order)   
        batch_indices = order[start:start + batch_size]
        batch = [source[index] for index in batch_indices]
        yield [source[index] for index in batch_indices]

# This is the iterator we use when we're evaluating our model. 
# It gives a list of batches that you can then iterate through.
def eval_iter(source, batch_size):
    batches = []
    dataset_size = len(source)
    start = -1 * batch_size
    order = list(range(dataset_size))
    random.shuffle(order)

    while start < dataset_size - batch_size:
        start += batch_size
        batch_indices = order[start:start + batch_size]
        batch = [source[index] for index in batch_indices]
        if len(batch) == batch_size:
            batches.append(batch)
        else:
            continue
        
    return batches

# The following function gives batches of vectors and labels, 
# these are the inputs to your model and loss function
def get_batch(batch):
    vectors = []
    labels = []
    for dict in batch:
        vectors.append(dict["text_index_sequence"])
        labels.append(dict["label"])
    return vectors, labels


## Evaluation

We'll be looking at accuracy as our evaluation metric.

In [0]:
# This function outputs the accuracy on the dataset, we will use it during training.
def evaluate(model, data_iter):
    model.eval()
    correct = 0
    total = 0
    for i in range(len(data_iter)):
        vectors, labels = get_batch(data_iter[i])
        vectors = torch.stack(vectors).squeeze()
        labels = torch.stack(labels).squeeze()

        output = model(vectors)
        
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
      
    return correct / float(total)

<br>

## Training Loop

The following function trains the model and reports model accuracy on the train and dev set every 5 epochs.

In [0]:
def training_loop(batch_size, num_epochs, model, loss_, optim, training_iter, dev_iter,
                  train_eval_iter, verbose=True):
    step = 0
    epoch = 0
    total_batches = int(len(training_set) / batch_size)
    accuracies = []
    while epoch <= num_epochs:
        model.train()
        vectors, labels = get_batch(next(training_iter)) 
        vectors = torch.stack(vectors).squeeze() # batch_size, seq_len
        labels = torch.stack(labels).squeeze()
        
    
        model.zero_grad()
        
        output = model(vectors)

        lossy = loss_(output, labels)
        lossy.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optim.step()
        

        if step % total_batches == 0:
            model.eval()
            if epoch % 5 == 0:
                train_acc = evaluate(model, train_eval_iter)
                eval_acc = evaluate(model, dev_iter)
                accuracies.append(eval_acc)
                if verbose:
                    print("Epoch %i; Step %i; Loss %f; Train acc: %f; Dev acc %f" 
                          %(epoch, step, lossy.item(),\
                            train_acc, eval_acc))
            epoch += 1
        step += 1
    
    best_dev = max(accuracies)
    print("Best dev accuracy is {}".format(best_dev))
    return best_dev

<br>

# Part 1: Implement DAN (40 points)

Following the [paper](https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf), implement the Deep Averaging Network (DAN).

Implementation details,
- Instead of using \code{tanh} activations however, use \code{ReLU}. 
- Make the number of layers a variable, not a fixed value.
- Make sure to implement word-dropout.

In [0]:
class DAN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size, 
                 batch_size, n_layers, drop_rate):
        super(DAN, self).__init__()
        
        self.embed = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embedding_size = embedding_dim
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
        self.n_layers = n_layers
        self.drop_rate = drop_rate
        
        """
        YOUR CODE GOES HERE
        """
        self.layers=nn.ModuleList()
        self.layers.append(nn.Linear(embedding_dim,hidden_size))
        for i in range(2,self.n_layers):
            self.append(nn.Linear(hidden_size.hidden_size))
        self.layers.append(nn.Linear(hidden_size,output_size))
        
        #print(self.layers)
        
        self.init_weights()
        
    
    def forward(self, input):
        """
        YOUR CODE GOES HERE
        """
        emb=self.embed(input)
        output=emb.mean(1)
        for i in range(self.n_layers-1):
            output=nn.functional.relu(self.layers[i](output))
            output=nn.functional.dropout(output,self.drop_rate)
        output= self.layers[self.n_layers-1](output)
        output= nn.functional.softmax(output)
        return output

    def init_hidden(self):
        h0 = torch.zeros(self.batch_size, self.hidden_size)
        return h0
    
    def init_weights(self):
        initrange = 0.1
        #lin_layers = [self.l1, self.decoder]
        #for i in range(1, self.n_layers):
        #    lin_layers.append(getattr(self, 'l{}'.format(i)))
        em_layer =  [self.embed]
        self.embed.weight.data.uniform_(-initrange, initrange)
        lin_layers = self.layers
        for layer in lin_layers:
            layer.weight.data.uniform_(-initrange, initrange)
            if layer in lin_layers:
                layer.bias.data.fill_(0)

### Train the model!

** Please use the hyperparameters and optimizer provided below. Do not make changes here. **

In [0]:
# Hyperparameters 
input_size = vocab_size
num_labels = 2
hidden_dim = 24
embedding_dim = 8
batch_size = 256
num_layers = 2
learning_rate = 0.001
drop_rate = 0.4
num_epochs = 30


# Build and initialize the model
dan = DAN(vocab_size, embedding_dim, hidden_dim, num_labels, batch_size, num_layers, drop_rate)
dan.init_weights()

# Define loss function and optimizer
loss = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(dan.parameters(), lr=learning_rate)

# Build data iterators
training_iter = data_iter(training_set, batch_size)
train_eval_iter = eval_iter(training_set[0:500], batch_size)
dev_iter = eval_iter(dev_set[:500], batch_size)

# Train the model
training_loop(batch_size, num_epochs, dan, loss, optimizer, training_iter, dev_iter, train_eval_iter)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 0; Step 0; Loss 0.693138; Train acc: 0.554688; Dev acc 0.531250
Epoch 5; Step 135; Loss 0.675479; Train acc: 0.675781; Dev acc 0.601562
Epoch 10; Step 270; Loss 0.509216; Train acc: 0.847656; Dev acc 0.730469
Epoch 15; Step 405; Loss 0.411843; Train acc: 0.937500; Dev acc 0.796875
Epoch 20; Step 540; Loss 0.354379; Train acc: 0.953125; Dev acc 0.785156
Epoch 25; Step 675; Loss 0.345872; Train acc: 0.972656; Dev acc 0.792969
Epoch 30; Step 810; Loss 0.337218; Train acc: 0.984375; Dev acc 0.796875
Best dev accuracy is 0.796875


0.796875

<br>

# Part 2: Hyperparameter tuning (40 points)

Tune the DAN for learning rate, number of layers, and drop-out rate. Select a range for each parameter and then do a random search over these hyperparameters, trying a minimum 5 permutations of hyperparameters. Report results and the best hyperparameters you found. Do you see any patterns in your results?

In [0]:
"""
YOUR CODE GOES HERE
"""


                
# Hyperparameters 2
input_size = vocab_size
num_labels = 2
hidden_dim = 24
embedding_dim = 8
batch_size = 256
num_layers = 4
learning_rate = 0.001
drop_rate = 0.4
num_epochs = 30




# Define loss function and optimizer
loss = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(dan.parameters(), lr=learning_rate)

# Build data iterators
training_iter = data_iter(training_set, batch_size)
train_eval_iter = eval_iter(training_set[0:500], batch_size)
dev_iter = eval_iter(dev_set[:500], batch_size)

# Train the model
training_loop(batch_size, num_epochs, dan, loss, optimizer, training_iter, dev_iter, train_eval_iter)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 0; Step 0; Loss 0.332679; Train acc: 0.980469; Dev acc 0.789062
Epoch 5; Step 135; Loss 0.328873; Train acc: 0.980469; Dev acc 0.800781
Epoch 10; Step 270; Loss 0.322728; Train acc: 0.984375; Dev acc 0.804688
Epoch 15; Step 405; Loss 0.322801; Train acc: 0.988281; Dev acc 0.785156
Epoch 20; Step 540; Loss 0.321557; Train acc: 0.992188; Dev acc 0.808594
Epoch 25; Step 675; Loss 0.320005; Train acc: 0.992188; Dev acc 0.792969
Epoch 30; Step 810; Loss 0.322944; Train acc: 0.992188; Dev acc 0.804688
Best dev accuracy is 0.80859375


0.80859375

In [0]:
"""
YOUR CODE GOES HERE
"""


                
# Hyperparameters 3
input_size = vocab_size
num_labels = 2
hidden_dim = 24
embedding_dim = 8
batch_size = 256
num_layers = 4
learning_rate = 0.0005
drop_rate = 0.4
num_epochs = 30




# Define loss function and optimizer
loss = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(dan.parameters(), lr=learning_rate)

# Build data iterators
training_iter = data_iter(training_set, batch_size)
train_eval_iter = eval_iter(training_set[0:500], batch_size)
dev_iter = eval_iter(dev_set[:500], batch_size)

# Train the model
training_loop(batch_size, num_epochs, dan, loss, optimizer, training_iter, dev_iter, train_eval_iter)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 0; Step 0; Loss 0.315523; Train acc: 0.996094; Dev acc 0.777344
Epoch 5; Step 135; Loss 0.330741; Train acc: 0.996094; Dev acc 0.792969
Epoch 10; Step 270; Loss 0.318331; Train acc: 0.996094; Dev acc 0.777344
Epoch 15; Step 405; Loss 0.321516; Train acc: 0.996094; Dev acc 0.792969
Epoch 20; Step 540; Loss 0.314178; Train acc: 0.996094; Dev acc 0.781250
Epoch 25; Step 675; Loss 0.319081; Train acc: 0.996094; Dev acc 0.777344
Epoch 30; Step 810; Loss 0.313781; Train acc: 0.996094; Dev acc 0.785156
Best dev accuracy is 0.79296875


0.79296875

In [0]:
"""
YOUR CODE GOES HERE
"""


                
# Hyperparameters 4
input_size = vocab_size
num_labels = 2
hidden_dim = 24
embedding_dim = 8
batch_size = 256
num_layers = 4
learning_rate = 0.002
drop_rate = 0.4
num_epochs = 30




# Define loss function and optimizer
loss = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(dan.parameters(), lr=learning_rate)

# Build data iterators
training_iter = data_iter(training_set, batch_size)
train_eval_iter = eval_iter(training_set[0:500], batch_size)
dev_iter = eval_iter(dev_set[:500], batch_size)

# Train the model
training_loop(batch_size, num_epochs, dan, loss, optimizer, training_iter, dev_iter, train_eval_iter)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 0; Step 0; Loss 0.333162; Train acc: 0.992188; Dev acc 0.765625
Epoch 5; Step 135; Loss 0.314002; Train acc: 0.992188; Dev acc 0.753906
Epoch 10; Step 270; Loss 0.317312; Train acc: 0.996094; Dev acc 0.750000
Epoch 15; Step 405; Loss 0.313887; Train acc: 0.996094; Dev acc 0.753906
Epoch 20; Step 540; Loss 0.317189; Train acc: 0.996094; Dev acc 0.753906
Epoch 25; Step 675; Loss 0.313786; Train acc: 0.996094; Dev acc 0.746094
Epoch 30; Step 810; Loss 0.317177; Train acc: 0.996094; Dev acc 0.757812
Best dev accuracy is 0.765625


0.765625

In [0]:
"""
YOUR CODE GOES HERE
"""


                
# Hyperparameters 5
input_size = vocab_size
num_labels = 2
hidden_dim = 24
embedding_dim = 8
batch_size = 256
num_layers = 6
learning_rate = 0.001
drop_rate = 0.4
num_epochs = 30




# Define loss function and optimizer
loss = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(dan.parameters(), lr=learning_rate)

# Build data iterators
training_iter = data_iter(training_set, batch_size)
train_eval_iter = eval_iter(training_set[0:500], batch_size)
dev_iter = eval_iter(dev_set[:500], batch_size)

# Train the model
training_loop(batch_size, num_epochs, dan, loss, optimizer, training_iter, dev_iter, train_eval_iter)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 0; Step 0; Loss 0.313531; Train acc: 0.996094; Dev acc 0.792969
Epoch 5; Step 135; Loss 0.317298; Train acc: 0.996094; Dev acc 0.781250
Epoch 10; Step 270; Loss 0.313275; Train acc: 1.000000; Dev acc 0.777344
Epoch 15; Step 405; Loss 0.313273; Train acc: 1.000000; Dev acc 0.773438
Epoch 20; Step 540; Loss 0.313366; Train acc: 1.000000; Dev acc 0.789062
Epoch 25; Step 675; Loss 0.313273; Train acc: 1.000000; Dev acc 0.785156
Epoch 30; Step 810; Loss 0.317177; Train acc: 1.000000; Dev acc 0.785156
Best dev accuracy is 0.79296875


0.79296875

In [0]:
"""
YOUR CODE GOES HERE
"""


                
# Hyperparameters 6
input_size = vocab_size
num_labels = 2
hidden_dim = 24
embedding_dim = 8
batch_size = 256
num_layers = 4
learning_rate = 0.001
drop_rate = 0.3
num_epochs = 30




# Define loss function and optimizer
loss = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(dan.parameters(), lr=learning_rate)

# Build data iterators
training_iter = data_iter(training_set, batch_size)
train_eval_iter = eval_iter(training_set[0:500], batch_size)
dev_iter = eval_iter(dev_set[:500], batch_size)

# Train the model
training_loop(batch_size, num_epochs, dan, loss, optimizer, training_iter, dev_iter, train_eval_iter)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 0; Step 0; Loss 0.317224; Train acc: 1.000000; Dev acc 0.804688
Epoch 5; Step 135; Loss 0.325070; Train acc: 1.000000; Dev acc 0.773438
Epoch 10; Step 270; Loss 0.317171; Train acc: 1.000000; Dev acc 0.796875
Epoch 15; Step 405; Loss 0.321182; Train acc: 1.000000; Dev acc 0.800781
Epoch 20; Step 540; Loss 0.313265; Train acc: 1.000000; Dev acc 0.785156
Epoch 25; Step 675; Loss 0.313305; Train acc: 1.000000; Dev acc 0.785156
Epoch 30; Step 810; Loss 0.317214; Train acc: 1.000000; Dev acc 0.796875
Best dev accuracy is 0.8046875


0.8046875

the best hypeparameters I used are: input_size = vocab_size
num_labels = 2
hidden_dim = 24
embedding_dim = 8
batch_size = 256
num_layers = 4
learning_rate = 0.001
drop_rate = 0.4
num_epochs = 30

I found out that increasing number of layers to 4 can improve the performance of the model; however, further increasing it to 6 results in a lower accuracy. 
Increasing the learning rate to 0.002 seem to have a negative impact on the performance of the model. 
Changing the drop rate didn't seem to make a hugh difference in my case.

<br>

# Part 3: short questions (20 points)

1. Name, and briefly describe, 3 other possible composition functions, instead of the DAN, you could use to build sentence representations.
2. Explain how dropout regularizes a model.
3. What are the shortcomings for training for a fixed number of epochs? Give an alternative.
4. Explain why you might use random search rather than grid search.

_Bonus (5 points): briefly describe the Nelder–Mead method and how you might use it to do hyperparamter tuning. What are the tradeoffs between using Nelder-Mead vs random search?_

1.
InferSent
InferSent is a sentence embeddings method that provides semantic representations for English sentences. It is trained on natural language inference data and generalizes well to many different tasks.
2.
Skip-Thought Vectors
In the skip-thought paper from 2015, the authors took the same intuition from the language model. However, in skip thought, instead of predicting the next word, we are predicting the previous and the next sentence. This gives the model more context for the sentence, thus we can build better representations of sentences. 
3.Learning Sentence Representations over Tree Structures for Target-dependent Classification
a reinforcement learning based approach, which
automatically induces target-specific sentence
representations over tree structures. The underlying model is a RNN encoder-decoder that
explores possible binary tree structures and a
reward mechanism that encourages structures
that improve performances on downstream
tasks. 


2.Dropout randomly ignores selected neurons during training so that their contribution to the activation of downstream neurons is temporally removed on the forward pass and any weight updates are not applied to the neuron on the backward pass. quoted partly from Jason Brownlee

3.The model might be able to continue improving after reaching the fixed number of epochs. An alternative: in Keras, there is an option of early stopping you can make when fitting a model to stop training once your validation accuracy flattens out, or starts to go back down

4.As random values are selected at each instance, it is highly likely that the whole of action space has been reached because of the randomness, which takes a huge amount of time to cover every aspect of the combination during grid search.  quoted from Kishan Maladkar

5.According to Wikipedia, "The Nelder–Mead method (also downhill simplex method, amoeba method, or polytope method) is a commonly applied numerical method used to find the minimum or maximum of an objective function in a multidimensional space. It is a direct search method (based on function comparison) and is often applied to nonlinear optimization problems for which derivatives may not be known. 
Trade off: However, the Nelder–Mead technique is a heuristic search method that can converge to non-stationary points on problems that can be solved by alternative methods."